In [6]:
import project_path

import pandas as pd
from sof_sa.utility_functions import connect2db, db_config

creds = db_config("..\\dev_dbconfig.yaml", 1)
engine = connect2db(**creds['TARGET_DB'])


{'TARGET_DB': {'DBUSER': 'zie', 'DBPASSWORD': 'Z01011010$$mysqlzie', 'DBHOST': 'localhost', 'DBPORT': 3306, 'DBNAME': 'sofanalysis'}}


In [5]:
# function to scrape data into staging db
import requests

url = 'https://www.globalbrainforce.com/blog/software-developer-salary-around-the-world/'

def scrape_data(url):
    """
    A function to scrape data from the web.
    Params :
        url - url for the web page to be scraped.
    Returns:
        Tables in the web page in a list of pandas dataframe objects
    """
    header = {
    "User-Agent" : "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/53>",
    "X-Requested-With" : "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)
    web_data = pd.read_html(r.text)
    return web_data

_data = scrape_data(url)[3]
_data

,Programming Language,Salary in USD
0,Perl,"$76,000"
1,Scala,"$76,000"
2,Go,"$74,000"
3,Rust,"$74,000"
4,Ruby,"$71,000"
5,Bash/Shell/PowerShell,"$65,000"
6,Objective-C,"$64,000"
7,Haskell,"$60,000"
8,Julia,"$59,000"
9,Python,"$59,000"


In [8]:
# Transformations
# Setting column names
_data.columns = ['progamming_language', 'salary_in_usd']

# Removing the $ sign from colmn salary_in_usd and setting type to float
_data['salary_in_usd'] = _data['salary_in_usd'].str.replace('$', '')
_data['salary_in_usd'] = _data['salary_in_usd'].astype(float)


Index(['progamming_language', 'salary_in_usd'], dtype='object')

In [ ]:
# Writing data to the database 
# Table name : programming_language_salary_staging
with engine.connect() as connection:
    _data.to_sql('programming_language_salary_staging', con=connection, if_exists='replace')

In [10]:
# Now working with csv files from stackoverflow survey
# loading in data 
df_2020 = pd.read_csv("..\\data\\survey_results_public_2020.csv")
df_2021 = pd.read_csv("..\\data\\survey_results_public_2021.csv")

# here we want to find :
#   the most popular language
#   the country with the most respondents
#   age and gender distibution amongst developers
#   how has employment changed over the pandemic
#   

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8
